In [26]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import LinearSVC
import sklearn

In [27]:
def inference(X, X_test, iterations, model):
    pred_list = []
    for i in range(iterations):
        X_train = X.sample(int(1*len(X)))

        #scale the data
        X_train, X_te = impute_and_scale(X_train, X_test)

        #encode categorical variables
        X_train, X_te = _ohe(X_train, X_te)
        
        pca = PCA(n_components=0.96, iterated_power=1)
        X_train = pca.fit_transform(X_train)
        X_te = pca.transform(X_te)

        #make predictions
        y_pred = model.predict_proba(X_te)[:,1]
        
        pred_list.append(y_pred)
    
    pred_df = pd.DataFrame(pred_list).T
    pred_df = pred_df.rank()
    pred_df["mean"] = pred_df.mean(axis=1)
    
    return pred_df

In [28]:
with open('/kaggle/working/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [29]:
predictions_df = inference(X, X_test, 50, model)

In [30]:
pred_df = pd.DataFrame(y_pred_list, columns=["pred_prob"])
pred_df["rank"] = pred_df.rank()
sub_df = pd.read_csv("../input/tabular-playground-series-aug-2022/sample_submission.csv")
sub_df["failure"] = predictions_df["mean"]
sub_df.to_csv('submission.csv', index=False)